# Squiggy Object-Oriented API Demo

This notebook demonstrates the new object-oriented API for working with POD5 and BAM files.

The OO API provides:
- Direct access to signals, alignments, and modifications
- Lazy iteration over reads  
- Customizable Bokeh plots
- No global state pollution

In [1]:
import squiggy
import numpy as np
from bokeh.plotting import show, output_notebook

# Enable Bokeh in notebook
output_notebook()

Loading BokehJS ...

## Opening Files

The new API uses `Pod5File` and `BamFile` classes instead of global state functions.

In [2]:
# Open POD5 file (no global state created)
pod5 = squiggy.Pod5File('../tests/data/yeast_trna_reads.pod5')
print(f"Loaded {len(pod5)} reads")

# Open BAM file
bam = squiggy.BamFile('../tests/data/yeast_trna_mappings.bam')
print(f"BAM has {len(bam.references)} references")

Loaded 180 reads
BAM has 164 references


## Accessing Read Data

Get a read and access its signal data directly.

In [3]:
# Get first read
read_id = pod5.read_ids[0]
read = pod5.get_read(read_id)

print(f"Read ID: {read.read_id}")
print(f"Signal length: {len(read.signal)} samples")
print(f"Sample rate: {read.sample_rate} Hz")
print(f"Duration: {len(read.signal) / read.sample_rate:.2f} seconds")

Read ID: 36d401b5-ba9c-4862-b003-7cd309ce5281
Signal length: 7962 samples
Sample rate: 4000 Hz
Duration: 1.99 seconds


## Signal Normalization

Access normalized signals using different methods.

In [4]:
# Get raw signal
raw_signal = read.signal

# Get normalized signals
znorm = read.get_normalized('ZNORM')  # Z-score normalization
median = read.get_normalized('MEDIAN')  # Median normalization
mad = read.get_normalized('MAD')  # Median absolute deviation

print(f"Raw signal: mean={raw_signal.mean():.2f}, std={raw_signal.std():.2f}")
print(f"ZNORM: mean={znorm.mean():.2f}, std={znorm.std():.2f}")
print(f"MEDIAN: mean={median.mean():.2f}, std={median.std():.2f}")
print(f"MAD: mean={mad.mean():.2f}, std={mad.std():.2f}")

Raw signal: mean=780.92, std=108.43
ZNORM: mean=-0.00, std=1.00
MEDIAN: mean=17.92, std=108.43
MAD: mean=0.15, std=0.90


## Getting Alignment Information

Extract base-to-signal mapping from BAM files.

In [5]:
# Get alignment for this read
alignment = read.get_alignment(bam)

if alignment:
    print(f"Sequence: {alignment.sequence[:50]}...")
    print(f"Number of bases: {len(alignment.bases)}")
    print(f"Aligned to: {alignment.chromosome}:{alignment.genomic_start}-{alignment.genomic_end}")
    print(f"Strand: {alignment.strand}")
    
    # Access individual base annotations
    print("\nFirst 10 bases:")
    for base in alignment.bases[:10]:
        signal_chunk = read.signal[base.signal_start:base.signal_end]
        print(f"  {base.base} (pos {base.position}): signal[{base.signal_start}:{base.signal_end}] mean={signal_chunk.mean():.1f}")
else:
    print("No alignment found (read may be unmapped or lack move table)")

Sequence: GTGCCTGGGCGTGTGGCGTAGTTGGTAGCGCACCCACCATGGGAGAGAGG...
Number of bases: 140
Aligned to: tRNA-Ala-AGC-1-1-uncharged:23-140
Strand: +

First 10 bases:
  G (pos 0): signal[0:84] mean=591.1
  T (pos 1): signal[84:108] mean=584.3
  G (pos 2): signal[108:132] mean=583.4
  C (pos 3): signal[132:156] mean=583.8
  C (pos 4): signal[156:180] mean=601.1
  T (pos 5): signal[180:204] mean=651.5
  G (pos 6): signal[204:252] mean=701.5
  G (pos 7): signal[252:276] mean=731.4
  G (pos 8): signal[276:300] mean=734.0
  C (pos 9): signal[300:324] mean=775.2


## Custom Analysis

Perform custom analysis on signal data.

In [6]:
if alignment:
    # Calculate mean signal per base
    base_signals = {}
    for base_ann in alignment.bases:
        base = base_ann.base
        signal_chunk = read.signal[base_ann.signal_start:base_ann.signal_end]
        
        if base not in base_signals:
            base_signals[base] = []
        base_signals[base].append(signal_chunk.mean())
    
    # Print statistics by base
    print("Mean signal by base:")
    for base in ['A', 'C', 'G', 'T']:
        if base in base_signals:
            signals = base_signals[base]
            print(f"  {base}: {np.mean(signals):.1f} ± {np.std(signals):.1f} (n={len(signals)})")

Mean signal by base:
  A: 800.4 ± 78.0 (n=30)
  C: 783.6 ± 95.6 (n=35)
  G: 765.9 ± 88.1 (n=39)
  T: 788.1 ± 97.5 (n=36)


## Plotting - Single Mode

Generate Bokeh plots that can be customized before display.

In [7]:
# Plot in SINGLE mode (raw signal)
fig = read.plot(mode='SINGLE', normalization='ZNORM')

# Customize the plot
fig.title.text = f"Custom Title: {read.read_id}"
fig.title.text_font_size = "16px"

# Display
show(fig)

## Plotting - Event-Aligned Mode

Plot with base annotations overlaid on signal.

In [8]:
# Plot with event alignment (requires BAM)
fig = read.plot(
    mode='EVENTALIGN',
    bam_file=bam,
    normalization='ZNORM',
    show_labels=True,
    theme='LIGHT'
)

# Customize the main plot
# Note: EVENTALIGN mode returns a layout with modification track + main plot
# Use .main_plot to access the main figure for customization
main_plot = fig.main_plot if hasattr(fig, 'main_plot') else fig
main_plot.title.text = f"Event-Aligned: {read.read_id}"

show(fig)

## Lazy Iteration

Efficiently iterate over reads without loading all into memory.

In [9]:
# Iterate over first 10 reads
print("Signal statistics for first 10 reads:\n")

for i, read in enumerate(pod5.iter_reads(limit=10)):
    signal = read.signal
    print(f"{i+1}. {read.read_id[:20]}... length={len(signal):,} mean={signal.mean():.1f}")

Signal statistics for first 10 reads:

1. 36d401b5-ba9c-4862-b... length=7,962 mean=780.9
2. 407164ac-c89a-4b42-a... length=9,763 mean=791.8
3. 42361bc3-ef8b-497f-8... length=10,731 mean=765.6
4. 99901b8e-c067-4aa3-9... length=9,665 mean=776.4
5. 3e35550e-e3c4-4fc3-b... length=9,022 mean=754.7
6. 76d1c615-9aaf-44d2-8... length=8,561 mean=799.6
7. 75de18f9-a365-4b0e-b... length=8,118 mean=790.8
8. 29d42eb8-f859-40b1-8... length=10,056 mean=751.6
9. 2951ec9a-e02f-4451-8... length=9,501 mean=795.2
10. 1faec289-0d39-4e2b-a... length=11,484 mean=843.4


## Batch Processing

Process multiple reads efficiently.

In [10]:
# Calculate signal length distribution
lengths = []

for read in pod5.iter_reads(limit=50):
    lengths.append(len(read.signal))

print(f"Signal length statistics (n={len(lengths)}):")
print(f"  Mean: {np.mean(lengths):,.0f} samples")
print(f"  Median: {np.median(lengths):,.0f} samples")
print(f"  Min: {np.min(lengths):,} samples")
print(f"  Max: {np.max(lengths):,} samples")

Signal length statistics (n=50):
  Mean: 10,188 samples
  Median: 9,697 samples
  Min: 7,603 samples
  Max: 24,542 samples


## Context Managers

Use `with` statements for automatic cleanup.

In [11]:
# Files are automatically closed when exiting context
with squiggy.Pod5File('../tests/data/yeast_trna_reads.pod5') as pod5_temp:
    with squiggy.BamFile('../tests/data/yeast_trna_mappings.bam') as bam_temp:
        read = pod5_temp.get_read(pod5_temp.read_ids[0])
        alignment = read.get_alignment(bam_temp)
        print(f"Read {read.read_id} has {len(alignment.bases)} bases" if alignment else "No alignment")

# Files are now closed automatically
print("Context managers exited, files closed")

Read 36d401b5-ba9c-4862-b003-7cd309ce5281 has 140 bases
Context managers exited, files closed


## Exporting Plots to HTML

Convert Bokeh figures to standalone HTML files.

In [12]:
# Generate plot
fig = read.plot(mode='SINGLE', normalization='ZNORM')

# Customize (SINGLE mode always returns a figure, not a layout)
fig.title.text = "My Custom Squiggle Plot"

# Convert to HTML
html = squiggy.figure_to_html(fig, title="My Squiggle Plot")

# Save to file
# with open('my_plot.html', 'w') as f:
#     f.write(html)

print(f"HTML length: {len(html)} characters")
print("(Uncomment to save to file)")

HTML length: 61195 characters
(Uncomment to save to file)


## Checking for Base Modifications

Query BAM files for modification information.

In [13]:
# Check if BAM contains base modifications
mod_info = bam.get_modifications_info()

print(f"Has modifications: {mod_info['has_modifications']}")
print(f"Modification types: {mod_info['modification_types']}")
print(f"Has probabilities: {mod_info['has_probabilities']}")
print(f"Reads sampled: {mod_info['sample_count']}")

Has modifications: True
Modification types: [17596, 17802, 'a', 'm']
Has probabilities: True
Reads sampled: 100


## Cleanup

Close files when done (if not using context managers).

In [14]:
pod5.close()
bam.close()
print("Files closed")

Files closed


## Summary

The new OO API provides:

✅ **Direct data access** - signals, alignments, modifications  
✅ **No global state** - multiple files can coexist  
✅ **Lazy loading** - efficient for large files  
✅ **Customizable plots** - modify Bokeh figures before display  
✅ **Context managers** - automatic cleanup with `with` statement  
✅ **Backward compatible** - legacy API still works for extension

For the Positron extension, the original API (`load_pod5()`, `plot_read()`) continues to work unchanged.